# 2.1 Instruction Fromatting Part 1

## Format the data

## Load the datasets

In [1]:
# imports
import pandas as pd

In [2]:
def load_dataset(file_path):
    with open(file_path, 'r') as f:
        data = pd.read_csv(file_path, encoding='utf-8-sig')
    return pd.DataFrame(data)

In [3]:
# load the datasets and process them accordingly
viet_data =load_dataset("../data/vietnamese_train_samples.csv")
fingpt_data = load_dataset("../data/fingpt_train_samples.csv")
synthesized_data = load_dataset("../data/synthetic_data_generate.csv")
viet_data_val = load_dataset("../data/vietnamese_validation_samples.csv")
fingpt_data_val = load_dataset("../data/fingpt_validation_samples.csv")

## Data Transformation
The library used (unsloth) requires the data to be in a certain format hence need to tranform the data into said format

In [4]:
vi_instruction = "Phân loại cảm xúc của câu này là gì? Hãy chọn câu trả lời từ {negative/neutral/positive}."

In [5]:
# transform the data to instruction format
# 1. fingpt data
for i in range(len(fingpt_data)):
    input = fingpt_data['input'][i]
    instruction = fingpt_data['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    fingpt_data.at[i, 'user_msg'] = user_msg

# 2. vietnamese data
for i in range(len(viet_data)):
    input = viet_data['sentence'][i]
    topic = viet_data['topic'][i]
    user_msg = "\"" + input + "\"" + " " + vi_instruction + " Đây là chủ đề: " + topic
    viet_data.at[i, 'user_msg'] = user_msg

# 3. synthesized data
for i in range(len(synthesized_data)):
    input = synthesized_data['input'][i]
    instruction = synthesized_data['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    synthesized_data.at[i, 'user_msg'] = user_msg

# 4. validation vietnamese data
for i in range(len(viet_data_val)):
    input = viet_data_val['sentence'][i]
    topic = viet_data_val['topic'][i]
    user_msg = "\"" + input + "\"" + " " + vi_instruction + " Đây là chủ đề: " + topic
    viet_data_val.at[i, 'user_msg'] = user_msg

# 5. validation fingpt data
for i in range(len(fingpt_data_val)):
    input = fingpt_data_val['input'][i]
    instruction = fingpt_data_val['instruction'][i]
    user_msg = "\"" + input + "\"" + " " + instruction
    fingpt_data_val.at[i, 'user_msg'] = user_msg


In [6]:
# Clean up the dataframes to only include necessary columns
fingpt_subset = pd.DataFrame({
    'user_msg': fingpt_data['user_msg'],
    'output': fingpt_data['output'],
})

viet_subset = pd.DataFrame({
    'user_msg': viet_data['user_msg'],
    'output': viet_data['sentiment'],
})

synth_subset = pd.DataFrame({
    'user_msg': synthesized_data['user_msg'],
    'output': synthesized_data['output'],
})

viet_val_subset = pd.DataFrame({
    'user_msg': viet_data_val['user_msg'],
    'output': viet_data_val['sentiment'],
    'language': 'vi',
})

fingpt_val_subset = pd.DataFrame({
    'user_msg': fingpt_data_val['user_msg'],
    'output': fingpt_data_val['output'],
    'language': 'eng',
})

In [7]:
combined_data = pd.concat([fingpt_subset, viet_subset, synth_subset], ignore_index=True)
combined_data = combined_data.sample(frac=1, random_state=1).reset_index(drop=True)
combined_data.head()

,user_msg,output
0,"""về trễ ."" Phân loại cảm xúc của câu này là gì...",negative
1,"""US Stocks Rise, but Growth Concerns Linger; M...",neutral
2,"""cần đa dạng hóa bài tập hơn ."" Phân loại cảm ...",negative
3,"""The smart money may be bailing out of Amazon,...",positive
4,"""nội dung môn học em nghĩ là rộng quá ạ ."" Phâ...",negative


In [8]:
combined_val_data = pd.concat([viet_val_subset, fingpt_val_subset], ignore_index=True)
combined_val_data = combined_val_data.sample(frac=1, random_state=1).reset_index(drop=True)
combined_val_data.head()

,user_msg,output,language
0,"""Is Now The Time To Look At Buying Avaya Holdi...",neutral,eng
1,"""'Mad Money' host Jim Cramer and the 'Squawk o...",negative,eng
2,"""mong thầy vẫn giữa phong độ như thế ;) ."" Phâ...",neutral,vi
3,"""nhiệt tình , tận tâm , hài hước , famous name...",positive,vi
4,"""The Department of Justice could sue Apple by ...",neutral,eng


In [9]:
combined_data.to_csv("../data/combined_data.csv", index=False, encoding='utf-8-sig')
combined_val_data.to_csv("../data/combined_val_data.csv", index=False, encoding='utf-8-sig')